In [136]:
from probabilidades import *
import csv
import pandas as pd

PDSexo = ProbDist(
    Sexo_M=4,
    Sexo_F=1,
)

PDIdades = ProbDist(
    {'Idade_A':4,
    'Idade_B':1,
    'Idade_C':0,
    'Idade_D':0}
)



pd_idade_sexo = joint(PDSexo, PDIdades, ' ')

with open('dados/trab_academicos.csv', 'r', encoding='utf8', newline='') as data:
    dados_brutos = pd.read_csv(data, delimiter=',')

In [137]:
#Nomalização de dados

dados = dados_brutos.rename(columns={'Quais dos seguintes ambientes você utiliza na edição de seus trabalhos?': 'ambientes',
                                    'Quais dos seguintes problemas você possui no momento da edição de trabalhos acadêmicos?': 'dificuldades',
                                    'Quais destas opções você gostaria que uma ferramenta de edição fornecesse como ajuda na edição de um trabalho acadêmico?': 'ajuda',
                                     'Com que frequência você realiza trabalhos acadêmicos?': 'frequencia',
                                     'Qual sua formação acadêmica?': 'nivel_ensino'
                                    })

for col in dados.columns:
    dados[col] = dados[col].str.lower()


In [138]:
#Funções de frequencia para montar as distribuições
def frequencia_dado(coluna):
    """Busca as frequencias para todos os valores possíveis da coluna
    param: Coluna a realizar a contagem da frequência
    return: Dicionário com a estrutura Valor:Freq
    """
    freq = {}
    for nivel in dados[coluna].unique():
        freq[nivel] = len(dados.loc[dados[coluna] == nivel])
    return freq

def frequencia_contida(coluna, data_freq):
    """Para perguntas com multiplas escolhas possíveis
    param: Coluna do dados em que deve ocorrer a busca
    param: Lista com os valores a serem buscadas as frêquencias
    """
    freq = {}
    for data in data_freq:
        freq[data] = len(dados.loc[dados[coluna].str.contains(data, case=False)])
    return freq


   


# Pergunta
1. Qual a probabilidade de uma pessoa que tenha respondido ao questionário ser do sexo masculino?
<br><span style="color: green;">R: Segundo o IBGE já que o questionário foi aplicado no Brasil, a probabilidade das pessoas que responderam o questionário ser do sexo masculino é de 48,5, isso com base no levantamento parcial para o ano de 2017.<span>

## Pergunta
2. Dado que uma pessoa indique que sua formação acadêmica é Ensino Superior Incompleto qual a probabilidade de ela ter problemas com Criar referências, Fazer citações ou Numeração de páginas?

In [139]:
freq_nivel = frequencia_dado('nivel_ensino')
dist_ensino = ProbDist(freq_nivel)

possiveis_problemas = list(set(';'.join(dados['dificuldades'].values).split(';')))

freq_problemas = frequencia_contida('dificuldades', possiveis_problemas)
dist_problemas = ProbDist(freq_problemas)

PDnivel_problema = joint(dist_ensino, dist_problemas, ' ')

#Predicados
def superior_incompleto(r):
    return 'superior incompleto' in r

def  dificuldades(r):
    return 'criar referências' in r or 'fazer citações' in r or 'numeração de páginas' in r


In [140]:
probabilidade_resposta2 = P(dificuldades, tal_que(superior_incompleto, PDnivel_problema))
print("A resposta da pergunta 2 é %.1f%%'" % (probabilidade_resposta2 * 100))

A resposta da pergunta 2 é 48.6%'


## Pergunta
3. Dado que uma pessoa indique que utiliza o ambiente Word qual a probabilidade de que o problema na edição de trabalhos acadêmicos seja Criação de sumários?

In [141]:
possiveis_ferramentas = list(set(';'.join(dados['ambientes'].values).split(';')))

freq_ambientes = frequencia_contida('ambientes', possiveis_ferramentas)

dist_ambientes = ProbDist(freq_ambientes)

PDambientes_problemas = joint(dist_ambientes, dist_problemas, ' ')

def word(r):
    return 'word' in r

def sumario(r):
    return 'criação de sumários' in r

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:19: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [142]:
probabilidade_resposta3 = P(sumario, tal_que(word, PDambientes_problemas))
print("A resposta da pergunta 3 é %.1f%%'" % (probabilidade_resposta3 * 100))
print(probabilidade_resposta3)


A resposta da pergunta 3 é 18.1%'
0.18148554701792902


## Pergunta
4. Dado que uma pessoa indique que utiliza o ambiente Word e seu problema de edição de trabalhos acadêmicos seja Criação de sumários qual a probabilidade de que ela gostaria que uma ferramenta de edição fornecesse como ajuda a Criação automática de sumários?

In [143]:
#Predicados
def word_criacao_sumario(r):
    return 'word' in r and 'criação de sumário' in r

def cricao_sumario(r):
    return 'criação automática de sumários' in r

possiveis_ajudas = list(set(';'.join(dados['ajuda'].values).split(';')))
dist_ajuda = frequencia_contida('ajuda', possiveis_ajudas)
PDAjuda = ProbDist(dist_ajuda)

PDAjuda_ambientes_problemas = joint(PDambientes_problemas, PDAjuda, ' ')

In [144]:
probabilidade_resposta4 = P(cricao_sumario, tal_que(word_criacao_sumario, PDAjuda_ambientes_problemas))
print("A resposta da pergunta 4 é %.1f%%'" % (probabilidade_resposta4 * 100))
print(probabilidade_resposta4)

A resposta da pergunta 4 é 11.5%'
0.11513157894736842


# Pergunta
5. Dado que uma pessoa indique que sua formação acadêmica é Ensino Fundamental Completo qual a probabilidade de ela utilizar Google Docs ou Word como ambiente de edição de trabalhos?

In [145]:
#Predicado
def google_word(r):
    return 'word' in r or 'google docs' in r

def ensino_fundamental(r):
    return 'ensino fundamental completo' in r

PDambientes_ensino = joint(dist_ambientes, dist_ensino, ' ')

In [146]:
probabilidade_resposta5 = P(google_word, tal_que(ensino_fundamental, PDambientes_ensino))
print("A resposta da pergunta 5 é %.1f%%'" % (probabilidade_resposta5 * 100))
print(probabilidade_resposta5)

A resposta da pergunta 5 é 77.8%'
0.7777777777777779


# Pergunta
6. Dado que uma pessoa indique que sua formação acadêmica é Ensino Superior Incompleto ou Ensino Superior Completo qual a probabilidade de ela não utilizar nem Google Docs e nem Word?

In [147]:
#Predicados

def superior(r):
    return 'ensino superior' in r

def not_google_word(r):
    return 'google docs' not in r and 'word' not in r

In [148]:
probabilidade_resposta6 = P(not_google_word,tal_que(superior, PDambientes_ensino))
print("A resposta da pergunta 6 é %.1f%%'" % (probabilidade_resposta6 * 100))
print(probabilidade_resposta6)

A resposta da pergunta 6 é 22.2%'
0.22222222222222207


# Pergunta
7. Dado que uma pessoa indique que utilize o ambiente Word qual os dois problemas com maiores probabilidades -- e quais seriam elas?

In [161]:
ferramenta_problema = joint(dist_ambientes, dist_problemas, '--')
word_problema = []
for i, y in ferramenta_problema.items():
    if 'word--' in i:
       word_problema.append((i,y))


In [163]:
ordenar_word_problema = sorted(word_problema, key=lambda tup: tup[1])

In [166]:
print(ordenar_word_problema[-2:])

[('word--fazer citações', 0.07626984126984121), ('word--criação de sumários', 0.07873015873015868)]
